In [1]:
from datasets import load_dataset

ds = load_dataset("malaysia-ai/common_voice_17_0")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00006.parquet:   0%|          | 0.00/145M [00:00<?, ?B/s]

data/train-00001-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

data/train-00002-of-00006.parquet:   0%|          | 0.00/146M [00:00<?, ?B/s]

data/train-00003-of-00006.parquet:   0%|          | 0.00/150M [00:00<?, ?B/s]

...

data/test-00000-of-00001.parquet:   0%|          | 0.00/84.5M [00:00<?, ?B/s]

data/dev-00000-of-00001.parquet:   0%|          | 0.00/72.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6689550 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/533642 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/507874 [00:00<?, ? examples/s]

In [4]:
df_train = ds['train'].to_pandas()

In [6]:
df_dev = ds['dev'].to_pandas()

In [15]:
!rm -rf mosaic-stt

In [16]:
from streaming import MDSWriter, LocalDataset
from tqdm import tqdm
import os

columns = {
    'audio_filename': 'str',
    'text': 'str',
}
hashes = 'sha1', 'xxh64'


with MDSWriter(out='mosaic-stt', columns=columns, compression=None, hashes=hashes) as out:
    
    for i in tqdm(range(len(df_train))):
        if not os.path.exists(df_train.iloc[i]['audio_filename']):
            continue
        
        t = df_train['sentence'].iloc[i]
            
        if not isinstance(t, str):
            continue
        
        if len(t) < 5:
            continue
            
        out.write({
            'audio_filename': df_train.iloc[i]['audio_filename'],
            'text': t,
        })
    
    for i in tqdm(range(len(df_dev))):
        if not os.path.exists(df_dev.iloc[i]['audio_filename']):
            continue
        
        t = df_dev['sentence'].iloc[i]
            
        if not isinstance(t, str):
            continue
        
        if len(t) < 5:
            continue
            
        out.write({
            'audio_filename': df_dev.iloc[i]['audio_filename'],
            'text': t,
        })

100%|████████████████████████████████| 507874/507874 [00:28<00:00, 18128.50it/s]
